
# CSE477 Lab 5: Clustering YouTube Comments — Step-by-Step (Colab Ready)

**What this notebook does**
- Loads your cleaned comments from Google Drive
- Converts text → TF‑IDF vectors
- Runs **K-Means** with elbow + helpful metrics
- Runs **DBSCAN** with guided `eps` tuning
- Gives **better visualizations** (term bars, 2D scatter, sample comments)
- Exports results to a CSV with cluster labels

**How to use (Google Colab)**
1. Upload this notebook to **Google Colab**.
2. Run the **Install & Setup** cell to mount Drive.
3. Make sure your file path is correct:  
   `DATA_PATH = '/content/drive/MyDrive/CSE477/cleaned_comments.csv'`
4. Run cells **top → bottom**. Fill the short reflections where asked.


In [ ]:

# --- Install & Setup (Colab) ---
# If you're on Colab, this will install any missing packages.
!pip -q install pandas numpy matplotlib seaborn scikit-learn umap-learn kneed

from google.colab import drive
drive.mount('/content/drive')

import os, ast, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from textwrap import shorten
plt.rcParams['figure.dpi'] = 120  # a bit crisper visuals

# >>>> EDIT ME if your file path is different <<<<
DATA_PATH = '/content/drive/MyDrive/CSE477/cleaned_comments.csv'

print("Using file:", DATA_PATH)
assert os.path.exists(DATA_PATH), "Path doesn't exist. Fix DATA_PATH above."



## Step 1 — Data Validation and Hypothesis
- Load your dataset (`.csv` or `.txt`).
- Report its shape with `df.shape`.
- **Critical Prompt #1** (answer just below the output):  
  Based on your number of comments, what do you expect about cluster **quality**?  
  Small datasets run, but do they yield meaningful topics?


In [ ]:

# Load dataset (CSV or TXT)
if DATA_PATH.lower().endswith(".csv"):
    df = pd.read_csv(DATA_PATH)
else:
    df = pd.read_table(DATA_PATH)

print(f"My dataset contains {df.shape[0]} comments and {df.shape[1]} columns.")
display(df.head())
print("Columns:", list(df.columns))



**Critical Prompt #1 (your short answer, 1–2 sentences):**  
> _Write your initial hypothesis about cluster quality here._



## Step 2 — Text → Vector (TF‑IDF)
Handles either `cleaned_tokens` (list/str of tokens) **or** `cleaned_text`.  
If neither is found, it tries to infer a text-like column automatically.


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

def build_corpus(df):
    if 'cleaned_tokens' in df.columns:
        def to_tokens(x):
            if isinstance(x, list):
                return x
            if isinstance(x, str):
                # try to parse list-like strings, else fallback to split
                try:
                    v = ast.literal_eval(x)
                    if isinstance(v, list):
                        return [str(t) for t in v]
                except Exception:
                    pass
                return x.split()
            return []
        corpus = df['cleaned_tokens'].apply(to_tokens).apply(lambda toks: ' '.join(toks)).tolist()
        text_col = 'cleaned_tokens'
    elif 'cleaned_text' in df.columns:
        corpus = df['cleaned_text'].astype(str).tolist()
        text_col = 'cleaned_text'
    else:
        # Try to infer
        cand = None
        for c in df.columns:
            if df[c].dtype == object:
                avg_len = df[c].astype(str).str.len().mean()
                if avg_len > 10:
                    cand = c; break
        if cand is None:
            raise ValueError("No usable text column found. Expected 'cleaned_tokens' or 'cleaned_text'.")
        corpus = df[cand].astype(str).tolist()
        text_col = cand
    return corpus, text_col

corpus, text_col = build_corpus(df)
print("Using text column:", text_col)

# TF-IDF
vectorizer = TfidfVectorizer(
    max_features=4000,       # you can raise this if you have a lot of data
    ngram_range=(1, 2),      # unigrams + bigrams to catch short phrases
    min_df=2,                # ignore ultra-rare tokens
    token_pattern=r"(?u)\b\w+\b"
)
X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names_out()

print("TF-IDF matrix shape:", X.shape)


In [ ]:

# Quick look: top TF-IDF terms globally
global_tfidf = np.asarray(X.sum(axis=0)).ravel()
topn = 20
top_idx = np.argsort(global_tfidf)[-topn:][::-1]

plt.figure(figsize=(8,5))
plt.barh(range(topn), global_tfidf[top_idx][::-1])
plt.yticks(range(topn), terms[top_idx][::-1])
plt.title("Top TF-IDF terms (global)")
plt.xlabel("TF-IDF sum")
plt.tight_layout()
plt.show()



## Step 3 — K-Means & Interpreting Results

### 3A. Elbow + Helpful Metrics
Besides inertia (elbow), we show:
- **Silhouette** (higher is better)
- **Calinski–Harabasz** (higher is better)
- **Davies–Bouldin** (lower is better)

This helps when the elbow looks ambiguous.


In [ ]:

from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Reduce dimensionality for fast metrics (and later plots)
svd_dim = min(50, max(2, X.shape[1]-1))
svd = TruncatedSVD(n_components=svd_dim, random_state=42)
X_svd = svd.fit_transform(X)

k_values = list(range(2, 9))
records, inertias = [], []

for k in k_values:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X)
    inertias.append(km.inertia_)
    try:
        sil = silhouette_score(X_svd, labels)
    except Exception:
        sil = float('nan')
    try:
        ch = calinski_harabasz_score(X_svd, labels)
    except Exception:
        ch = float('nan')
    try:
        db = davies_bouldin_score(X_svd, labels)
    except Exception:
        db = float('nan')
    records.append({
        'k': k,
        'inertia': km.inertia_,
        'silhouette (SVD)': sil,
        'calinski_harabasz (SVD)': ch,
        'davies_bouldin (SVD)': db,
    })

metrics_df = pd.DataFrame(records)
display(metrics_df.style.background_gradient(axis=0))

# Elbow plot
plt.figure(figsize=(6,4))
plt.plot(k_values, inertias, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.tight_layout()
plt.show()

# Silhouette plot
plt.figure(figsize=(6,4))
plt.plot(k_values, metrics_df['silhouette (SVD)'], marker='o')
plt.xlabel('k')
plt.ylabel('Silhouette (SVD space)')
plt.title('Silhouette vs k')
plt.tight_layout()
plt.show()

# Try automatic knee detection (optional)
auto_k = None
try:
    from kneed import KneeLocator
    kl = KneeLocator(k_values, inertias, curve='convex', direction='decreasing')
    auto_k = kl.elbow
    print("Auto-selected k (elbow):", auto_k)
except Exception as e:
    print("KneeLocator not available/failed:", e)

# Choose k
optimal_k = int(auto_k) if auto_k is not None else 3
print("Using k =", optimal_k)



### 3B. Run K-Means + Inspect Cluster Keywords


In [ ]:

kmeans = KMeans(n_clusters=int(optimal_k), random_state=42, n_init=10)
df['kmeans_label'] = kmeans.fit_predict(X)

# Top terms per cluster (printed + bar plot)
for i in range(int(optimal_k)):
    center = kmeans.cluster_centers_[i]
    top_idx = center.argsort()[-10:][::-1]
    print(f"Cluster {i}: " + ", ".join([f"{terms[j]}({center[j]:.3f})" for j in top_idx]))

    plt.figure(figsize=(8,4))
    plt.barh(range(10), center[top_idx][::-1])
    plt.yticks(range(10), terms[top_idx][::-1])
    plt.title(f"Cluster {i}: top TF-IDF terms")
    plt.tight_layout()
    plt.show()



### 3C. Visualize K-Means (2D)
We use **TruncatedSVD(2)** (works well for sparse TF‑IDF) to project to 2D.


In [ ]:

svd2 = TruncatedSVD(n_components=2, random_state=42)
X_2d = svd2.fit_transform(X)

plt.figure(figsize=(8,6))
scatter = plt.scatter(X_2d[:,0], X_2d[:,1], c=df['kmeans_label'], alpha=0.6)
plt.title('K-Means Cluster Visualization (SVD 2D)')
plt.xlabel('Component 1'); plt.ylabel('Component 2')
plt.colorbar(scatter, label='Cluster')
plt.tight_layout()
plt.show()

# Show 3 representative comments closest to each cluster center (in 2D)
import numpy as np
examples = {}
for i in range(int(optimal_k)):
    idxs = np.where(df['kmeans_label'].values==i)[0]
    if len(idxs)==0:
        continue
    centroid = X_2d[idxs].mean(axis=0)
    dists = np.linalg.norm(X_2d[idxs] - centroid, axis=1)
    top = idxs[np.argsort(dists)[:3]]
    print(f"\nCluster {i} sample comments:")
    for t in df.iloc[top][text_col].tolist():
        print("-", shorten(str(t).replace("\n"," "), width=200))



**Critical Prompt #2:**  
State your chosen **k** and **justify it** (refer to elbow/metrics).  
Give **one good** cluster and **one bad/confusing** cluster, with reasoning.



## Step 4 — DBSCAN (Comparative Analysis)

Key ideas:
- `eps`: neighborhood radius  
- `min_samples`: minimum points to form a cluster (start with 5)

We **scale** the sparse TF‑IDF with `StandardScaler(with_mean=False)` and try multiple `eps` values.


In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Scale sparse matrix directly
X_scaled = StandardScaler(with_mean=False).fit_transform(X)

eps_candidates = [0.3, 0.5, 0.7, 0.9, 1.2]
results = []
labels_map = {}

for eps in eps_candidates:
    dbscan = DBSCAN(eps=eps, min_samples=5, n_jobs=-1)
    labels = dbscan.fit_predict(X_scaled)
    labels_map[eps] = labels

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = int((labels == -1).sum())

    # silhouette only if at least 2 clusters
    try:
        from sklearn.metrics import silhouette_score
        sil = silhouette_score(X_svd, labels) if n_clusters > 1 else float('nan')
    except Exception:
        sil = float('nan')

    results.append({'eps': eps, 'min_samples': 5, 'clusters': n_clusters, 'noise_points': n_noise, 'silhouette (SVD)': sil})

res_df = pd.DataFrame(results)
display(res_df.style.background_gradient(axis=0))

# Choose best eps by silhouette (fallback to 0.5 if all NaN)
if res_df['silhouette (SVD)'].notna().any():
    best_eps = float(res_df.sort_values(['silhouette (SVD)'], ascending=False).iloc[0]['eps'])
else:
    best_eps = 0.5

print("Chosen eps:", best_eps)
dbscan_final = DBSCAN(eps=best_eps, min_samples=5, n_jobs=-1)
final_labels = dbscan_final.fit_predict(X_scaled)
df['dbscan_label'] = final_labels


In [ ]:

# Visualize DBSCAN on the same 2D projection (X_2d)
plt.figure(figsize=(8,6))
scatter = plt.scatter(X_2d[:,0], X_2d[:,1], c=df['dbscan_label'], alpha=0.6)
plt.title(f'DBSCAN Visualization (eps={best_eps}, -1 is Noise)')
plt.xlabel('Component 1'); plt.ylabel('Component 2')
plt.colorbar(scatter, label='Cluster/Noise')
plt.tight_layout()
plt.show()

def counts(d):
    u, c = np.unique(d, return_counts=True)
    return dict(zip(u, c))

print("K-Means cluster counts:", counts(df['kmeans_label'].values))
print("DBSCAN cluster/noise counts:", counts(df['dbscan_label'].values))



**Critical Prompt #3:**  
Compare **K-Means vs DBSCAN** on your data. Which gave more **useful insight**, and why?  
Describe a scenario where **DBSCAN** would be **better** than K-Means for comment analysis.



## Step 5 — Final Reflection & Application

**Critical Prompt #4:**  
How did your dataset's characteristics (size, topic, language style) influence results?  
If an airline or marketing company used this on customer feedback, what's the **single most important lesson** from this lab you'd tell them?



## Export Results
Save a CSV with the original text + labels so you can analyze clusters elsewhere.


In [ ]:

# Export labeled data
keep_cols = [text_col] + [c for c in ['kmeans_label','dbscan_label'] if c in df.columns]
out_path = '/content/CSE477_lab5_clustered_comments.csv'
df[keep_cols].to_csv(out_path, index=False)
print("Saved:", out_path)



## (Optional) UMAP 2D (Often prettier than SVD/PCA)
Run this if you want an alternative 2D view.


In [ ]:

try:
    import umap
    umap_2d = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1).fit_transform(X_svd)
    plt.figure(figsize=(8,6))
    sc = plt.scatter(umap_2d[:,0], umap_2d[:,1], c=df['kmeans_label'], alpha=0.6)
    plt.title('UMAP 2D — colored by K-Means')
    plt.colorbar(sc, label='Cluster')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("UMAP not available:", e)
